# Feature extraction

This is the final notebook explaining the methodology for text feature extraction. We will explain each of the different features extracted and then we will explain the process and methodology to extract all the features and create a new dataset.

## Index

- [1. Features](#1.-Feature-explanation)

 - [1.1. Complexity features](#1.1.-Complexity-features)
 - [1.2. Stylometric features](#1.2.-Stylometric-features)


- [2. Requisites](#2.-Requisites)


- [3. Feature extraction for training](#3.-Feature-extraction-for-training)


- [4. Feature extraction function for predictions](#4.-Feature-extraction-function-for-predictions)

## 1. Feature explanation

On this section we will explain the features that we are going to extract from the News Headline and News Content text.These features are language-independent, for example, they do not consider specific terms from a language, in this case spanish.

Our objective is to extract features based on high-level structures. To accomplish this objective, we are going to extract features from 2 categories: Complexity and Stylometric

### 1.1. Complexity features

The objective of these features is to capture te overall intricacy of the news, in sentence and word level. To achive this, we use metrics like average word size, words count per sentence and type token ratio:

**words**: Number of words

**Sents**: Number of sentences

**avg_words_sentence**: Average words per sentence

**avg_word_size**: Average word size

**avg_syllables_word**: Average syllables per word

**unique_words**: Hapaxes or unique words that only appears once in a text

**ttr**: Type token ratio

### Bonus ###

Spanish readability tests:

**huerta_score**: Fernández Huerta's redability score (Reading comprehension of the text), spanish adaptation of the Flesch equation

&nbsp;&nbsp; $$Readability = 206.84 - 0.60 \times Average Syllables Word - 1.02 \times Average Words Sentence$$ &nbsp;&nbsp;

**szigriszt_score**: Szigriszt Pazos perspicuity score (Legibility and clarity of the text), a modern spanish adaptation of the Flesch equation.

&nbsp;&nbsp; $$Perspicuity = 206.835 - \frac{62.3 \times TotalSyllables}{Words} - \frac{Words}{Sentences}$$ &nbsp;&nbsp;

### 1.2. Stylometric features
For stylometric or lexical features, we use NLP techniques to extract grammatical and lexical information for each text. We are using Spacy POS tagging techniques to track different word style frequencies:

**mltd**: Measure of Textual Lexical Diversity, based on McCarthy and Jarvis (2010).

**upper_case_ratio**: Uppercase letters to all letters ratio

**entityratio**: Ratio of named Entities to the text size

**quotes**: Number of quotes

**quotes_ratio**: Ratio of quotes marks to text size

**propn_ratio**: Proper Noun tag frequency

**noun_ratio**: Noun tag frequency

**pron_ratio**: Pronoun tag frequency

**adp_ratio**: Adposition tag frequency

**det_ratio**: Determinant tag frequency

**punct_ratio**: Punctuation tag frequency

**verb_ratio**: Verb tag frequency

**adv_ratio**: Adverb tag frequency

**sym_ratio**: Symbol tag frequency

### 2. Requisites

*For Python 3 installations use ___!pip3 install___ and ___python3 *___

[NLTK package](https://pypi.org/project/nltk/)

`!pip install nltk`

`import nltk`


[Spacy spanish package](https://spacy.io/models/es)

`!pip install spacy`

`python -m spacy download es_core_news_lg`

`import spacy`


[lexical_diversity package](https://pypi.org/project/lexical-diversity/)

`!pip install lexical-diversity`

`from lexical_diversity import lex_div as ld`


[Syltippy](https://github.com/nur-ag/syltippy)

Syltippy is a simple, user friendly word syllabization package for spanish language with no additional dependencies.

`!pip install syltippy`

`from syltippy import syllabize`

## 3. Feature extraction for training

In [1]:
# Tried several syllabizers for spanish and this is the chosen solution. Believe me, i spent a whole day.
# I had to replace all symbols, punctuations and it includes accentuation from other languages like ä, à, etc...
# It's a bit inconsistent with words from others languages, acronyms and abreviations. However it performs really well for our case!!!

def get_nsyllables(text):
    from syltippy import syllabize

    text = text.replace(r"*NUMBER*", "número")
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")
    text = re.sub(r'\d+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(r'[^ \nA-Za-z0-9ÁÉÍÓÚÑáéíóúñ/]+', '', text)
    
    n_syllables = len(syllabize(text)[0])
    
    return n_syllables

In [4]:
%%time

import itertools
import pandas as pd
import nltk
import spacy
import re
from nltk import FreqDist
from sklearn.preprocessing import LabelEncoder
from lexical_diversity import lex_div as ld
pd.options.display.max_columns = None

nlp = spacy.load('es_core_news_lg')

df = pd.read_csv('../data/corpus_spanish_v3.csv')

labelencoder = LabelEncoder()
df['Label'] = labelencoder.fit_transform(df['Category'])

# empty lists and df
df_features = pd.DataFrame()
list_text = []
list_sentences = []
list_words = []
list_words_sent = []
list_word_size = []
list_avg_syllables_word = []
list_unique_words = []
list_ttr = []
list_huerta_score = []
list_szigriszt_score = []
list_mltd = []
list_entity_ratio = []
list_upper_case_ratio = []
list_quotes = []
list_quotes_ratio = []
list_propn_ratio = [] 
list_noun_ratio = []
list_adp_ratio = []
list_det_ratio = []
list_punct_ratio = []
list_pron_ratio = []
list_verb_ratio = []
list_adv_ratio = []
list_sym_ratio = []

list_headline = []
list_words_h = []
list_word_size_h = []
list_avg_syllables_word_h = []
list_ttr_h = []
list_mltd_h = []
list_unique_words_h = []

# df iteration
for n, row in df.iterrows():
    
    ## headline ##
    headline = df['Headline'].iloc[n]
    headline = re.sub(r"http\S+", "", headline)
    headline = re.sub(r"http", "", headline)
    headline = re.sub(r"@\S+", "", headline)
    headline = re.sub("\n", " ", headline)
    headline = re.sub(r"(?<!\n)\n(?!\n)", " ", headline)
    headline = headline.replace(r"*NUMBER*", "número")
    headline = headline.replace(r"*PHONE*", "número")
    headline = headline.replace(r"*EMAIL*", "email")
    headline = headline.replace(r"*URL*", "url")
    headline = headline.lower()
    doc_h = nlp(headline)
    
    list_tokens_h = []
    list_tags_h = []

    for sentence_h in doc_h.sents:
        for token in sentence_h:
            list_tokens_h.append(token.text)

    fdist_h = FreqDist(list_tokens_h)
    syllables_h = get_nsyllables(headline)
    words_h = len(list_tokens_h)
    
    # headline complexity features
    avg_word_size_h = round(sum(len(word) for word in list_tokens_h) / words_h, 2)
    avg_syllables_word_h = round(syllables_h / words_h, 2)
    unique_words_h = round((len(fdist_h.hapaxes()) / words_h) * 100, 2)
    ttr_h = round(ld.ttr(list_tokens_h) * 100, 2)
    mltd_h = round(ld.mtld(list_tokens_h), 2)
    
    ## text content##   
    text = df['Text'].iloc[n]  
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http", "", text)
    text = re.sub("\n", " ", text)
    text = text.replace(r"*NUMBER*", "número")
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")
    
    # to later calculate upper case letters ratio
    alph = list(filter(str.isalpha, text))
    text_lower = text.lower()
    doc = nlp(text_lower)

    list_tokens = []
    list_pos = []
    list_tag = []
    list_entities = []
    sents = 0
    
    for entity in doc.ents:
        list_entities.append(entity.label_)

    for sentence in doc.sents:
        sents += 1
        for token in sentence:
            list_tokens.append(token.text)
            list_pos.append(token.pos_)
            list_tag.append(token.tag_)
    
    # Calculate entities, pos, tag, freq, syllables, words and quotes
    entities = len(list_entities)
    n_pos = nltk.Counter(list_pos)
    n_tag = nltk.Counter(list_tag)
    fdist = FreqDist(list_tokens)
    syllables = get_nsyllables(text)
    words = len(list_tokens)
    quotes = n_tag['PUNCT__PunctType=Quot']

    # complexity features
    avg_word_sentence = round(words / sents, 2)
    avg_word_size = round(sum(len(word) for word in list_tokens) / words, 2)
    avg_syllables_word = round(syllables / words, 2)
    unique_words = round((len(fdist.hapaxes()) / words) * 100, 2)
    ttr = round(ld.ttr(list_tokens) * 100, 2)
    mltd = round(ld.mtld(list_tokens), 2)

    # readability spanish test
    huerta_score = round(206.84 - (60 * avg_syllables_word) - (1.02 * avg_word_sentence), 2)
    szigriszt_score = round(206.835 - ((62.3 * syllables) / words) - (words / sents), 2)

    # stylometric features
    upper_case_ratio = round(sum(map(str.isupper, alph)) / len(alph) * 100, 2)
    entity_ratio = round((entities / words) * 100, 2)
    quotes_ratio = round((quotes / words) * 100, 2)
    propn_ratio = round((n_pos['PROPN'] / words) * 100 , 2)
    noun_ratio = round((n_pos['NOUN'] / words) * 100, 2) 
    adp_ratio = round((n_pos['ADP'] / words) * 100, 2)
    det_ratio = round((n_pos['DET'] / words) * 100, 2)
    punct_ratio = round((n_pos['PUNCT'] / words) * 100, 2)
    pron_ratio = round((n_pos['PRON'] / words) * 100, 2)
    verb_ratio = round((n_pos['VERB'] / words) * 100, 2)
    adv_ratio = round((n_pos['ADV'] / words) * 100, 2)
    sym_ratio = round((n_tag['SYM'] / words) * 100, 2)
    
    # appending on lists
    # headline
    list_headline.append(headline)
    list_words_h.append(words_h)
    list_word_size_h.append(avg_word_size_h)
    list_avg_syllables_word_h.append(avg_syllables_word_h)
    list_unique_words_h.append(unique_words_h)
    list_ttr_h.append(ttr_h)
    list_mltd_h.append(mltd_h)
    
    # text
    list_text.append(text_lower)
    list_sentences.append(sents)
    list_words.append(words)
    list_words_sent.append(avg_word_sentence)
    list_word_size.append(avg_word_size)
    list_avg_syllables_word.append(avg_syllables_word)
    list_unique_words.append(unique_words)
    list_ttr.append(ttr)
    list_huerta_score.append(huerta_score)
    list_szigriszt_score.append(szigriszt_score)
    list_mltd.append(mltd)
    list_entity_ratio.append(entity_ratio)
    list_upper_case_ratio.append(upper_case_ratio)
    list_quotes.append(quotes)
    list_quotes_ratio.append(quotes_ratio)
    list_propn_ratio.append(propn_ratio)
    list_noun_ratio.append(noun_ratio)
    list_adp_ratio.append(adp_ratio)
    list_det_ratio.append(det_ratio)
    list_punct_ratio.append(punct_ratio)
    list_pron_ratio.append(pron_ratio)
    list_verb_ratio.append(verb_ratio)
    list_adv_ratio.append(adv_ratio)
    list_sym_ratio.append(sym_ratio)
    
# dataframe
df_features['topic'] = df['Topic']
df_features['text'] = list_text
df_features['headline'] = list_headline

# headline
df_features['words_h'] = list_words_h
df_features['word_size_h'] = list_word_size_h
df_features['avg_syllables_word_h'] = list_avg_syllables_word_h
df_features['unique_words_h'] = list_unique_words_h
df_features['ttr_h'] = list_ttr_h
df_features['mltd_h'] = list_mltd_h

# text
df_features['sents'] = list_sentences
df_features['words'] = list_words
df_features['avg_words_sent'] = list_words_sent
df_features['avg_word_size'] = list_word_size
df_features['avg_syllables_word'] = list_avg_syllables_word
df_features['unique_words'] = list_unique_words
df_features['ttr'] = list_ttr
df_features['mltd'] = list_mltd
df_features['huerta_score'] = list_huerta_score
df_features['szigriszt_score'] = list_szigriszt_score
df_features['upper_case_ratio'] = list_upper_case_ratio
df_features['entity_ratio'] = list_entity_ratio
df_features['quotes'] = list_quotes
df_features['quotes_ratio'] = list_quotes_ratio
df_features['propn_ratio'] = list_propn_ratio
df_features['noun_ratio'] = list_noun_ratio
df_features['adp_ratio'] = list_adp_ratio
df_features['det_ratio'] = list_det_ratio
df_features['punct_ratio'] = list_punct_ratio
df_features['pron_ratio'] = list_pron_ratio
df_features['verb_ratio'] = list_verb_ratio
df_features['adv_ratio'] = list_adv_ratio
df_features['sym_ratio'] = list_sym_ratio

df_features['label'] = df['Label']

df_features.to_csv('../data/spanish_corpus_features_v6.csv', encoding = 'utf-8', index = False)

CPU times: user 5min 21s, sys: 3.12 s, total: 5min 24s
Wall time: 5min 24s


In [5]:
df_features

,topic,text,headline,words_h,word_size_h,avg_syllables_word_h,unique_words_h,ttr_h,mltd_h,sents,words,avg_words_sent,avg_word_size,avg_syllables_word,unique_words,ttr,mltd,huerta_score,szigriszt_score,upper_case_ratio,entity_ratio,quotes,quotes_ratio,propn_ratio,noun_ratio,adp_ratio,det_ratio,punct_ratio,pron_ratio,verb_ratio,adv_ratio,sym_ratio,label
0,Science,la nasa recupera el contacto con un satélite d...,la nasa recupera el contacto con un satélite d...,16,5.38,2.50,87.50,93.75,71.68,16,479,29.94,4.76,2.04,31.52,43.22,56.66,53.90,49.96,3.87,2.71,0,0.00,4.80,24.43,17.75,15.66,7.10,1.67,8.35,3.13,0.00,1
1,Economy,amlo aceleraría el consumo y el crecimiento ec...,amlo aceleraría el consumo y el crecimiento ec...,11,5.27,2.55,81.82,90.91,33.88,5,206,41.20,4.58,1.89,33.98,49.03,57.83,51.42,47.69,4.35,5.83,2,0.97,9.22,22.82,15.53,16.50,11.17,0.49,6.31,0.97,0.00,1
2,Sport,al borde de un colapso nervioso quedó el hábil...,compañero de james se ‘calvea’ y le juega pesa...,12,3.75,1.67,100.00,100.00,0.00,13,368,28.31,4.23,1.70,41.58,53.26,64.43,75.96,72.72,2.60,3.80,7,1.90,7.34,16.58,15.22,13.32,13.04,4.35,8.15,3.53,0.54,0
3,Politics,"mediante pupitrazo de último minuto anoche, el...",dian gravará este año a los niños que recojan ...,12,4.58,1.92,100.00,100.00,0.00,8,269,33.62,4.75,1.92,40.15,53.53,65.16,57.35,53.71,2.02,1.49,4,1.49,2.23,22.30,19.33,15.24,10.04,0.74,6.69,3.35,0.00,0
4,Politics,muy temprano esta mañana el expresidente y aho...,uribe asegura que insultó 358 guerrilleros,6,6.17,2.50,100.00,100.00,0.00,9,260,28.89,4.18,1.59,49.62,61.54,93.74,81.97,78.75,1.81,2.69,7,2.69,5.77,17.31,8.46,10.77,20.00,5.38,10.38,5.00,0.77,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3969,Sport,fifa levanta sanción a messi para regresar a l...,fifa levanta sanción a messi para regresar a l...,10,5.10,2.30,80.00,90.00,28.00,7,342,48.86,4.77,1.95,38.30,51.17,83.37,40.00,36.29,4.75,5.56,4,1.17,8.19,18.71,19.30,15.20,8.77,2.63,8.77,2.34,0.00,1
3970,Education,el debate sobre los deberes llega al congreso ...,el debate sobre los deberes llega al congreso ...,11,4.73,2.00,81.82,90.91,33.88,23,1036,45.04,4.30,1.75,24.23,35.23,56.55,55.90,52.47,5.13,2.32,48,4.63,3.96,19.59,14.96,14.77,13.71,1.45,8.49,2.70,0.48,1
3971,Society,como un paso decisivo en la descongestión judi...,tuiteros emberracados podrán juzgar algunos de...,6,7.67,3.00,100.00,100.00,0.00,9,447,49.67,4.46,1.84,43.18,53.47,92.41,45.78,42.46,0.88,1.34,8,1.79,1.57,22.37,15.66,11.41,9.84,3.36,8.95,4.25,0.00,0
3972,Science,muy preocupada se declaró la secretaría de sal...,ojear pantallas ajenas en transmilenio ya es u...,9,5.44,2.33,100.00,100.00,0.00,19,475,25.00,4.22,1.75,42.11,53.26,73.15,76.34,72.71,2.07,2.32,6,1.26,3.16,16.00,14.74,10.95,11.79,7.58,11.79,3.37,0.63,0


## 4. Feature extraction function for predictions

To make predictions we need to extracte features from a given news headline and news text content. So we are going to pack the code above to extract the features for our predictions

In [10]:
%%time

import pandas as pd
import nltk
import spacy
import re
from nltk import FreqDist
from lexical_diversity import lex_div as ld

def get_news_features(headline, text):
    
    nlp = spacy.load('es_core_news_lg')

    ## headline ##
    headline = re.sub(r"http\S+", "", headline)
    headline = re.sub(r"http", "", headline)
    headline = re.sub(r"@\S+", "", headline)
    headline = re.sub("\n", " ", headline)
    headline = re.sub(r"(?<!\n)\n(?!\n)", " ", headline)
    headline = headline.replace(r"*NUMBER*", "número")
    headline = headline.replace(r"*PHONE*", "número")
    headline = headline.replace(r"*EMAIL*", "email")
    headline = headline.replace(r"*URL*", "url")
    headline_new = headline.lower()
    doc_h = nlp(headline_new)

    list_tokens_h = []
    list_tags_h = []

    for sentence_h in doc_h.sents:
        for token in sentence_h:
            list_tokens_h.append(token.text)

    fdist_h = FreqDist(list_tokens_h)
    syllables_h = get_nsyllables(headline)
    words_h = len(list_tokens_h)

    # headline complexity features
    avg_word_size_h = round(sum(len(word) for word in list_tokens_h) / words_h, 2)
    avg_syllables_word_h = round(syllables_h / words_h, 2)
    unique_words_h = round((len(fdist_h.hapaxes()) / words_h) * 100, 2)
    mltd_h = round(ld.mtld(list_tokens_h), 2)
    ttr_h = round(ld.ttr(list_tokens_h) * 100, 2)

    ## text content##     
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http", "", text)
    text = re.sub("\n", " ", text)
    text = text.replace(r"*NUMBER*", "número")
    text = text.replace(r"*PHONE*", "número")
    text = text.replace(r"*EMAIL*", "email")
    text = text.replace(r"*URL*", "url")

    # to later calculate upper case letters ratio
    alph = list(filter(str.isalpha, text))
    text_lower = text.lower()
    doc = nlp(text_lower)

    list_tokens = []
    list_pos = []
    list_tag = []
    list_entities = []
    sents = 0

    for entity in doc.ents:
        list_entities.append(entity.label_)

    for sentence in doc.sents:
        sents += 1
        for token in sentence:
            list_tokens.append(token.text)
            list_pos.append(token.pos_)
            list_tag.append(token.tag_)

    # Calculate entities, pos, tag, freq, syllables, words and quotes
    entities = len(list_entities)
    n_pos = nltk.Counter(list_pos)
    n_tag = nltk.Counter(list_tag)
    fdist = FreqDist(list_tokens)
    syllables = get_nsyllables(text)
    words = len(list_tokens)
    quotes = n_tag['PUNCT__PunctType=Quot']

    # complexity features
    avg_word_sentence = round(words / sents, 2)
    avg_word_size = round(sum(len(word) for word in list_tokens) / words, 2)
    avg_syllables_word = round(syllables / words, 2)
    unique_words = round((len(fdist.hapaxes()) / words) * 100, 2)
    ttr = round(ld.ttr(list_tokens) * 100, 2)

    # readability spanish test
    huerta_score = round(206.84 - (60 * avg_syllables_word) - (1.02 * avg_word_sentence), 2)
    szigriszt_score = round(206.835 - ((62.3 * syllables) / words) - (words / sents), 2)

    # stylometric features
    mltd = round(ld.mtld(list_tokens), 2)
    upper_case_ratio = round(sum(map(str.isupper, alph)) / len(alph) * 100, 2)
    entity_ratio = round((entities / words) * 100, 2)
    quotes_ratio = round((quotes / words) * 100, 2)
    propn_ratio = round((n_pos['PROPN'] / words) * 100 , 2)
    noun_ratio = round((n_pos['NOUN'] / words) * 100, 2) 
    pron_ratio = round((n_pos['PRON'] / words) * 100, 2)
    adp_ratio = round((n_pos['ADP'] / words) * 100, 2)
    det_ratio = round((n_pos['DET'] / words) * 100, 2)
    punct_ratio = round((n_pos['PUNCT'] / words) * 100, 2)
    verb_ratio = round((n_pos['VERB'] / words) * 100, 2)
    adv_ratio = round((n_pos['ADV'] / words) * 100, 2)
    sym_ratio = round((n_tag['SYM'] / words) * 100, 2)

    # create df_features
    df_features = pd.DataFrame({'words_h': words_h, 'word_size_h': [avg_word_size_h],'avg_syllables_word_h': [avg_syllables_word_h],
                                'unique_words_h': [unique_words_h], 'ttr_h': ttr_h, 'mltd_h': [mltd_h], 'sents': sents, 'words': words,
                                'avg_words_sent': [avg_word_sentence], 'avg_word_size': [avg_word_size], 
                                'avg_syllables_word': avg_syllables_word, 'unique_words': [unique_words], 
                                'ttr': [ttr], 'huerta_score': [huerta_score], 'szigriszt_score': [szigriszt_score],
                                'mltd': [mltd], 'upper_case_ratio': [upper_case_ratio], 'entity_ratio': [entity_ratio],
                                'quotes': quotes, 'quotes_ratio': [quotes_ratio], 'propn_ratio': [propn_ratio], 
                                'noun_ratio': [noun_ratio], 'pron_ratio': [pron_ratio], 'adp_ratio': [adp_ratio],
                                'det_ratio': [det_ratio], 'punct_ratio': [punct_ratio], 'verb_ratio': [verb_ratio],
                                'adv_ratio': [adv_ratio], 'sym_ratio': [sym_ratio]})
    
    return df_features

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 36.7 µs


In [11]:
headline = input('Insert news headline:')
text = input('Insert news content:')

get_news_features(text, headline)

Insert news headline:El Gobierno ha presentado hoy al Niño de Schrödinger, que va y no va al colegio
insert news content:La ministra de Educación y Formación Profesional, Isabel Celaá, ha presentado esta mañana al Niño de Schrödinger, fruto de un proyecto en el que han colaborado varias universidades españolas y que viene a resolver el problema de la vuelta a los colegios en plena ola de contagios por coronavirus.  «Va y no va al colegio y está expuesto al virus pero al mismo tiempo no lo está», ha explicado Celaá, insistiendo en que se trata de «una paradoja avalada científicamente».  La ministra ha mostrado a los medios al niño, cuyo nombre es Fernando Campos Leza, describiéndolo como «un alumno perfectamente sano y normal que ahora mismo, estando aquí con nosotros, está al mismo tiempo en casa, donde permanecerá mientras vaya al colegio con normalidad junto al resto de niños de Schrödinger».  A partir de mañana y hasta el inicio del nuevo curso escolar, los padres deberán adaptar a 

,words_h,avg_word_size_h,avg_syllables_word,unique_words_h,mltd_h,sents,words,avg_word_sentence,avg_word_size,unique_words,ttr,huerta_score,szigriszt_score,mltd,upper_case_ratio,entity_ratio,quotes,quotes_ratio,propn_ratio,noun_ratio,pron_ratio,adp_ratio,det_ratio,punct_ratio,verb_ratio,adv_ratio,sym_ratio
0,258,4.4,1.47,35.66,65.99,1,17,17.0,3.76,76.47,88.24,101.3,98.22,40.46,6.35,11.76,0,0.0,17.65,5.88,0.0,17.65,5.88,5.88,17.65,11.76,0.0
